Правила:



*   Домашнее задание оценивается в 10 баллов.
*   Можно использовать без доказательства любые результаты, встречавшиеся на лекциях или семинарах по курсу, если получение этих результатов не является вопросом задания.
*  Можно использовать любые свободные источники с *обязательным* указанием ссылки на них.
*  Плагиат не допускается. При обнаружении случаев списывания, 0 за работу выставляется всем участникам нарушения, даже если можно установить, кто у кого списал.
*  Старайтесь сделать код как можно более оптимальным. В частности, будет штрафоваться использование циклов в тех случаях, когда операцию можно совершить при помощи инструментов библиотек, о которых рассказывалось в курсе.

В этом домашнем задании вам предстоит построить классификатор текстов.

Будем предсказывать эмоциональную окраску твиттов о коронавирусе.



In [3]:
import numpy as np
import pandas as pd
from typing import  List
import matplotlib.pyplot as plt
import seaborn as sns
from string import punctuation

In [4]:
df = pd.read_csv('content/tweets_coronavirus.csv', encoding='latin-1')
df.sample(4)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
4738,9553,54505,Georgia,19-03-2020,FYI: Covid-19 Update\r\r\nIf you are a grocery...,Extremely Positive
23500,32505,77457,"Vancouver, Canada",05-04-2020,Victoria helps with food production for 1st ti...,Positive
11599,17865,62817,United Kingdom,21-03-2020,The Supermarkets are loving it - so many despe...,Negative
15395,22535,67487,Australia,23-03-2020,@stevebhyve @abcnews yes but have you heard th...,Extremely Positive


Для каждого твитта указано:


*   UserName - имя пользователя, заменено на целое число для анонимности
*   ScreenName - отображающееся имя пользователя, заменено на целое число для анонимности
*   Location - местоположение
*   TweetAt - дата создания твитта
*   OriginalTweet - текст твитта
*   Sentiment - эмоциональная окраска твитта (целевая переменная)



## Задание 1 Подготовка (0.5 балла)

Целевая переменная находится в колонке `Sentiment`.  Преобразуйте ее таким образом, чтобы она стала бинарной: 1 - если у твитта положительная эмоциональная окраска и 0 - если отрицательная.

In [5]:
df["Sentiment"].value_counts() # узнаём, какие варианты окраски есть

Positive              11422
Negative               9917
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [6]:
df["Sentiment"] = df["Sentiment"].apply(lambda x: 1 if x in ["Positive", "Extremely Positive"] else 0)
df.head(10)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,1
1,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,1
2,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,1
3,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",0
4,3804,48756,"ÃÂT: 36.319708,-82.363649",16-03-2020,As news of the regionÃÂs first confirmed COV...,1
5,3805,48757,"35.926541,-78.753267",16-03-2020,Cashier at grocery store was sharing his insig...,1
6,3807,48759,"Atlanta, GA USA",16-03-2020,Due to COVID-19 our retail store and classroom...,1
7,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy th...",0
8,3810,48762,"Pitt Meadows, BC, Canada",16-03-2020,"Due to the Covid-19 situation, we have increas...",1
9,3811,48763,Horningsea,16-03-2020,#horningsea is a caring community. LetÃÂs AL...,1


Сбалансированы ли классы?

In [7]:
df["Sentiment"].value_counts()

1    18046
0    15398
Name: Sentiment, dtype: int64

**Ответ:** Разница между численностью классов, конечно, есть, но незначительная, поэтому классы сбалансированы

Выведете на экран информацию о пропусках в данных. Если пропуски присутствуют заполните их строкой 'Unknown'.

In [8]:
df.isnull().any().any()

True

In [9]:
# пропуски есть, заполним их строкой "Unknown"
df = df.fillna("Unknown")
df.head(5)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,1
1,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,1
2,3802,48754,Unknown,16-03-2020,My food stock is not the only one which is emp...,1
3,3803,48755,Unknown,16-03-2020,"Me, ready to go at supermarket during the #COV...",0
4,3804,48756,"ÃÂT: 36.319708,-82.363649",16-03-2020,As news of the regionÃÂs first confirmed COV...,1


Разделите данные на обучающие и тестовые в соотношении 7 : 3 и `random_state=0`

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3, random_state=0)

## Задание 2 Токенизация (3 балла)

Постройте словарь на основе обучающей выборки и посчитайте количество встреч каждого токена с использованием самой простой токенизации - деления текстов по пробельным символам и приведение токенов в нижний регистр

In [11]:
td = {}
tweets = train["OriginalTweet"].apply(lambda x: x.lower().split())
for i in tweets:
    for j in i:
        if j in td.keys():
            td[j] += 1
        else:
            td[j] = 1
td

{'why': 654,
 'we': 3787,
 'still': 763,
 'want': 487,
 'to': 23373,
 'buy': 748,
 'so': 1837,
 'much': 482,
 'stuff': 106,
 'during': 1908,
 'quarantine': 154,
 'https://t.co/1m881cwfuv': 1,
 '#shopping': 121,
 '#covid_19': 1660,
 '#online': 40,
 'with': 4063,
 'driving': 77,
 'even': 682,
 'more': 2008,
 'usage': 22,
 'a': 11737,
 'strong': 93,
 'strategy': 33,
 'is': 7383,
 'critical': 156,
 'now': 1349,
 'then': 435,
 'ever': 179,
 'luckily': 8,
 'and': 14684,
 'april': 163,
 '16': 36,
 'webinar': 65,
 'will': 2726,
 'dive': 10,
 'into': 666,
 'growing': 84,
 'app': 52,
 'awareness': 32,
 'targeting': 9,
 'high': 408,
 'lifetime': 4,
 'value': 79,
 'users': 19,
 '@canon_india': 16,
 'i': 5340,
 'am': 424,
 'very': 418,
 'happy..': 1,
 'great': 409,
 'job': 242,
 'by': 2236,
 '#canonforcommunity': 5,
 'https://t.co/z0intks34x': 1,
 'the': 26815,
 'u.s': 8,
 'national': 123,
 'debt': 68,
 'likely': 170,
 'exceed': 14,
 '$30': 7,
 'trillion': 21,
 'after': 886,
 'unprecedented': 89,
 

Какой размер словаря получился?

In [12]:
len(td)

79755

Выведите 10 самых популярных токенов с количеством встреч каждого из них. Объясните, почему именно эти токены в топе

In [13]:
sorted(td.items(), key=lambda x: x[1], reverse=True)[:10]

[('the', 26815),
 ('to', 23373),
 ('and', 14684),
 ('of', 13012),
 ('a', 11737),
 ('in', 11198),
 ('for', 8566),
 ('#coronavirus', 8223),
 ('is', 7383),
 ('are', 7050)]

**Ответ:** Именно эти токены в топе, потому что они (все кроме одного) представляют из себя важные грамматические структуры английского языка, встречающиеся практически в любом предложении: артикли, предлоги и формы глагола to be. Кроме них в топ входит '#coronavirus', но и это логично - выборка ведь состоит только из твитов о коронавирусе, поэтому данный хэштег ожидаемо очень часто используем.

Удалите стоп-слова из словаря и выведите новый топ-10 токенов (и количество встреч) по популярности.  Что можно сказать  о нем?

In [14]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/striker_isp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
for i in stopwords.words('english'):
    if i in td.keys():
        td.pop(i)
sorted(td.items(), key=lambda x: x[1], reverse=True)[:10]

[('#coronavirus', 8223),
 ('prices', 3891),
 ('food', 3820),
 ('grocery', 3469),
 ('supermarket', 3288),
 ('people', 3175),
 ('covid-19', 3173),
 ('store', 3155),
 ('#covid19', 2471),
 ('&amp;', 2314)]

**Ответ:** Все грамматические слова были удалены. Хэштег, разумеется, остался. Из остального, часто упоминаются prices, food, grocery, supermarket, store - что указывает на проблемы с ценами в магазинах, когда люди подняли спрос на повседневные товары, закупившись ими в начале пандемии. Также, разумеется, часто встречается covid-19 и #covid-19, по аналогии с уже присутствовавшим хэштегом. Частота использования токена people тоже логична, ведь твиты о коронавирусе - это твиты о чем-то глобальном, затрагивающем много людей, и о действиях этого множества людей. Также присутствует токен \&amp;, который предположительно представляет из себя эмодзи или некий символ

Также выведите 20 самых непопулярных слов (если самых непопулярных слов больше выведите любые 20 из них) Почему эти токены непопулярны, требуется ли как-то дополнительно работать с ними?

In [16]:
sorted(td.items(), key=lambda x: x[1])[:20]

[('https://t.co/1m881cwfuv', 1),
 ('happy..', 1),
 ('https://t.co/z0intks34x', 1),
 ('mnuchinã\x82â\x92s', 1),
 ('brink.', 1),
 ('https://t.co/jmobv8z0u0', 1),
 ("university's", 1),
 ('teaching.)', 1),
 ('@catholicpres', 1),
 ('@catholicuniv', 1),
 ('https://t.co/evqby035wf', 1),
 ('https://t.co/riqrhxxeim', 1),
 ('@@ballardspahrll', 1),
 ('#aca', 1),
 ('easier...take', 1),
 ('ã\x82â\x93necessaryã\x82â\x94', 1),
 ('https://t.co/0fmsmlgepm', 1),
 ('husted:', 1),
 ('irishman', 1),
 ('#happystpatricksday!', 1)]

**Ответ:** Это, в основном, ссылки, юзернеймы и т.д. И конечно они не популярны, ведь это могут быть ссылки на никому не известные ресурсы или условно тэги юзернеймов друзей с 0.00001 подписчиком, которые делаются один раз одним человеком. В целом, для целей машинного обучения эти ссылки можно попытаться убрать, ведь они (по крайней мере, очевидным образом) не влияют на эмоциональную окраску твитов. С другой стороны, приведу пример: если какая-то ссылка в твите A указывает на другой твит B, который можно явно охарактеризовать как негативный, а в твите A нет ничего кроме ссылки и слова "Yes", то охарактеризовать этот твит как негативный можно только при обладании этой ссылкой. Поэтому тут все зависит от того, как мы решим действовать - включать в анализ твиты-ответы как выражающие согласную с изначальным твитом эмоцию или нет.

Теперь воспользуемся токенайзером получше - TweetTokenizer из библиотеки nltk. Примените его и посмотрите на топ-10 популярных слов. Чем он отличается от топа, который получался раньше? Почему?

In [17]:
from nltk.tokenize import TweetTokenizer

t = TweetTokenizer()
td_tok = {}
for i in train["OriginalTweet"]:
    tweet = t.tokenize(i)
    for j in tweet:
        if j in td_tok.keys():
            td_tok[j] += 1
        else:
            td_tok[j] = 1
sorted(td_tok.items(), key=lambda x: x[1], reverse=True)[:10]

[('the', 24337),
 ('.', 24118),
 ('to', 22933),
 (',', 17571),
 ('and', 14354),
 ('of', 12904),
 ('a', 11045),
 ('in', 10573),
 ('?', 9524),
 ('for', 8228)]

**Ответ:** Этот топ отличается тем, что включает в себя знаки препинания, потому что, видимо, данный токенизатор выделяет пунктуацию как отдельные токены. Ну а вхождение знаков препинания в топ объяснимо так же, как и с грамматическими словами - некоторые из знаков пунктуации так или иначе присутствуют почти в любом предложении.

Удалите из словаря стоп-слова и пунктуацию, посмотрите на новый топ-10 слов с количеством встреч, есть ли теперь в нем что-то не похожее на слова?

In [18]:
from string import punctuation

for i in stopwords.words('english'):
    if i in td_tok.keys():
        td_tok.pop(i)
        
for i in punctuation:
    if i in td_tok.keys():
        td_tok.pop(i)

sorted(td_tok.items(), key=lambda x: x[1], reverse=True)[:10]

[('Â', 7415),
 ('\x82', 7311),
 ('19', 7167),
 ('#coronavirus', 7143),
 ('I', 5235),
 ('\x92', 4372),
 ('prices', 4281),
 ('COVID', 4218),
 ('food', 3795),
 ('store', 3691)]

**Ответ:** Некоторые токены те же, что и в токенизации по пробелам выше после удаления стоп-слов. Также добавилось число 19 (предположительно, от твитов, которые писали "covid 19" через пробел), добавился COVID заглавными буквами (ранее такого токена не было, так как при токенизации твиты были приведены в нижний регистр) и добавилось местоимение I, встречающееся почти в любом предложении, автор которого хочет что-то сказать о себе (ранее этого местоимения в топе не было, возможно потому что оно не отделялось от конструкций по типу I'm или I'd и воспринималось с ними как одно целое). Еще появились токены Â, \x82 и \x92, первое из которых точно, а второе и третье предположительно, являются символами

Удалите из словаря токены из одного символа, с позицией в таблице Unicode 128 и более (`ord(x) >= 128`)

Выведите топ-10 самых популярных и топ-20 непопулярных слов. Чем полученные топы отличаются от итоговых топов, полученных при использовании токенизации по пробелам? Что теперь лучше, а что хуже?

In [19]:
for i in td_tok.copy().keys():
    if len(i) == 1 and ord(i) >= 128:
        td_tok.pop(i)

sorted(td_tok.items(), key=lambda x: x[1], reverse=True)[:10]

[('19', 7167),
 ('#coronavirus', 7143),
 ('I', 5235),
 ('prices', 4281),
 ('COVID', 4218),
 ('food', 3795),
 ('store', 3691),
 ('supermarket', 3373),
 ('grocery', 3083),
 ('people', 3047)]

In [20]:
sorted(td_tok.items(), key=lambda x: x[1])[:20]

[('https://t.co/1m881CwFUv', 1),
 ('https://t.co/Z0intkS34x', 1),
 ('MnuchinÃ', 1),
 ('https://t.co/JmoBv8z0U0', 1),
 ("University's", 1),
 ('@CatholicPres', 1),
 ('@CatholicUniv', 1),
 ('https://t.co/EvQby035wF', 1),
 ('https://t.co/rIQrhxxeIM', 1),
 ('@BallardSpahrLL', 1),
 ('#training', 1),
 ('#aca', 1),
 ('https://t.co/0FmSmlGePM', 1),
 ('Irishman', 1),
 ('#HappyStPatricksDay', 1),
 ('Guiness', 1),
 ('https://t.co/18V0PYHwb7', 1),
 ('https://t.co/LS0g86i8PU', 1),
 ('https://t.co/oFVkejfF23', 1),
 ('https://t.co/Xoodgd2uLx', 1)]

**Ответ:** Полученные топы отличаются от итоговых при токенизации по пробелам тем, что они учитывают регистр, способны отделить слово от привязанной к нему сокращенной грамматической структуры ('m, 'd) - это о топ-10 самых частых. А топ-20 самых нечастых по сути отличается незначительно - в нем разве что стало больше хэштегов.
Я бы сказал, что топ-10 лучших стал лучше с твит-токенизатором (плюс: упомянутая способность отделить слово от сокращенной грамматической структуры; а минус, что разными считаются одинаковые слова в разном регистре, в рамках нашей задачи превращается в плюс - ведь, например, написание слова заглавными буквами выражает определенную эмоцию). И топ-20 непопулярных, пожалуй, тоже был улучшен твит-токенизатором, так как теперь в нем выделено больше хэштегов.

Выведите топ-10 популярных хештегов с количеством встреч. Что можно сказать о них?

In [21]:
htd = {}
tweets = train["OriginalTweet"].apply(lambda x: x.lower().split())
for i in tweets:
    for j in i:
        if j[0] == '#':
            if j in htd.keys():
                htd[j] += 1
            else:
                htd[j] = 1
sorted(htd.items(), key=lambda x: x[1], reverse=True)[:10]

[('#coronavirus', 8223),
 ('#covid19', 2471),
 ('#covid_19', 1660),
 ('#covid2019', 915),
 ('#toiletpaper', 691),
 ('#socialdistancing', 427),
 ('#coronacrisis', 423),
 ('#covid?19', 418),
 ('#coronaviruspandemic', 245),
 ('#pandemic', 232)]

**Ответ:** Все хэштеги либо являются непосредственными синонимами (или различными написаниями) названия вируса covid-19 и связанной с ним пандемии, либо отсылают к кейсам пандемии covid-19 (известный спрос на туалетную бумагу при кризисах (в частности, при пандемии covid-19), социальная дистанция)

То же самое проделайте для ссылок на сайт https://t.co Сравнима ли популярность ссылок с популярностью хештегов? Будет ли информация о ссылке на конкретную страницу полезна?

In [22]:
ltd = {}
tweets = train["OriginalTweet"].apply(lambda x: x.lower().split())
for i in tweets:
    for j in i:
        if j[:12] == 'https://t.co':
            if j in ltd.keys():
                ltd[j] += 1
            else:
                ltd[j] = 1
sorted(ltd.items(), key=lambda x: x[1], reverse=True)[:10]

[('https://t.co/oxa7swtond', 5),
 ('https://t.co/gp3eusapl8', 4),
 ('https://t.co/deftrui1pfã\x82â', 3),
 ('https://t.co/wrlhyzizaa', 3),
 ('https://t.co/kuwipf1kqw', 3),
 ('https://t.co/zjnrx6dkkn', 3),
 ('https://t.co/3gbbdpdjat', 3),
 ('https://t.co/e2znxajpre', 3),
 ('https://t.co/catkegayoy', 3),
 ('https://t.co/g63rp042ho', 3)]

**Ответ:** # Популярность ссылок абсолютно не сравнима с популярностью хэштегов. И это логично, ведь суть хэштего в массовом обсуждении тем, в то время как ссылки зачастую публикуют на отдельные точечные ресурсы. Поэтому информация о ссылке на конкретную страницу не будет полезна, ведь одинаковых ссылок слишком мало в выборке для должного обучения на них. Хотя, конечно, так же как и в примере с твитом-ответом, содержимое ссылки может помочь в определении эмоциональной окраски твита, если оно является эмоционально окрашенным.

Используем опыт предыдущих экспериментов и напишем собственный токенайзер, улучшив TweetTokenizer. Функция tokenize должна:



*   Привести текст в нижний регистр
*   Применить TweetTokenizer для изначального выделения токенов
*   Удалить стоп-слова, пунктуацию, токены из одного символа, с позицией в таблице Unicode 128 и более и ссылки на t.co



In [23]:
def custom_tokenizer(text):
    text = text.lower()
    
    t = TweetTokenizer()
    tokens = t.tokenize(text)
    
    for i in stopwords.words('english'):
        while i in tokens:
            tokens.remove(i)
    for i in punctuation:
        while i in tokens:
            tokens.remove(i)
    for i in tokens.copy():
        if len(i) == 1 and ord(i) >= 128:
            tokens.remove(i)
    for i in tokens.copy():
        if i[:12] == 'с':
            tokens.remove(i)
    
    return tokens


In [24]:
custom_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext')

['sample', 'text', '@sample_text', '#sampletext']

## Задание 3 Векторизация текстов (2 балла)

Обучите CountVectorizer с использованием custom_tokenizer в качестве токенайзера. Как размер полученного словаря соотносится с размером изначального словаря из начала задания 2?

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(tokenizer=custom_tokenizer)
cvft = cv.fit_transform(train["OriginalTweet"])

print(len(cv.vocabulary_))

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


45308


**Ответ:** Полученный словарь сильноо меньше изначального словаря из задания 2... почти в 2 раза. 

Посмотрим на какой-нибудь конкретный твитт:

In [26]:
ind = 9023
train.iloc[ind]['OriginalTweet'], train.iloc[ind]['Sentiment']

('Nice one @SkyNews lets not panic but show ppl in france queueing for food!!! #CoronavirusOutbreak #COVID2019 brainless!! Ffs',
 0)

Автор твитта не доволен ситуацией с едой во Франции и текст имеет резко негативную окраску.

Примените обученный CountVectorizer для векторизации данного текста, и попытайтесь определить самый важный токен и самый неважный токен (токен, компонента которого в векторе максимальна/минимальна, без учета 0). Хорошо ли они определились, почему?

In [27]:
print(cv.transform([train.iloc[ind]["OriginalTweet"]]))

  (0, 1833)	1
  (0, 2046)	1
  (0, 16658)	1
  (0, 20553)	1
  (0, 26665)	1
  (0, 27042)	1
  (0, 27233)	1
  (0, 31579)	1
  (0, 34079)	1
  (0, 34688)	1
  (0, 35257)	1
  (0, 36413)	1
  (0, 37229)	1
  (0, 39919)	1


**Ответ:** Определилось совсем не хорошо (не определилось). У всех токенов компонента = 1. А не определились, потому что все токены встречаются в предложении только по 1 разу

Теперь примените TfidfVectorizer и  определите самый важный/неважный токены. Хорошо ли определились, почему?

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(tokenizer=custom_tokenizer)
tvft = tv.fit_transform(train["OriginalTweet"])

print(tv.transform([train.iloc[ind]["OriginalTweet"]]))
print(tv.get_feature_names_out()[tv.transform([train.iloc[ind]["OriginalTweet"]]).argmin()]) # самый неважный
print(tv.get_feature_names_out()[tv.transform([train.iloc[ind]["OriginalTweet"]]).argmax()]) # самый важный

  (0, 39919)	0.24406492401820118
  (0, 37229)	0.35434556333216544
  (0, 36413)	0.2550064610386691
  (0, 35257)	0.14759470306326164
  (0, 34688)	0.16306722855395073
  (0, 34079)	0.2542729176022132
  (0, 31579)	0.30121070136861045
  (0, 27233)	0.32047314046539654
  (0, 27042)	0.11328893069250721
  (0, 26665)	0.3244741764513268
  (0, 20553)	0.3867641170466375
  (0, 16658)	0.31042634466284263
  (0, 2046)	0.16825192939361902
  (0, 1833)	0.22584265007428544
###covid-19
brainless


**Ответ:** Самый важный - brainless. Самый неважный - ###covid-19. Определились хорошо, потому что в tf-idf важна не только частота в одном твите, но и частота во всех твитах в выборке, что и делает значения различными.

Найдите какой-нибудь положительно окрашенный твитт, где TfidfVectorizer хорошо (полезно для определения окраски) выделяет важный токен, поясните пример.

*Подсказка:* явно положительные твитты можно искать при помощи положительных слов (good, great, amazing и т. д.)

In [29]:
train[train['OriginalTweet'].apply(lambda x: 'amazing' in x) & (train['Sentiment'] == 1)].sample(15)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
11674,17953,62905,Essex,21-03-2020,Thank you so much @waitrose you have been amaz...,1
29904,40517,85469,India,09-04-2020,@Canon_India Coming together for a amazing cau...,1
23318,32289,77241,"Dublin City, Ireland",04-04-2020,Healthcare workers are amazing during this #CO...,1
6074,11177,56129,"Ottawa Hills, Ohio",19-03-2020,#SickLeave #Petition | #Kroger is the largest ...,1
21299,29778,74730,"Morrisville, NC",01-04-2020,We've got newly updated info about online shop...,1
10468,16499,61451,Unknown,21-03-2020,This Covid-19 is really an amazing ÃÂselecti...,1
13355,20006,64958,"Los Angeles, CA",22-03-2020,"""This particular edition of Writer's Lounge is...",1
3567,8135,53087,"Surrey, Sussex or Cornwall",18-03-2020,Spare a thought for our amazing doctors and nu...,1
19919,28079,73031,"Newbury, Berkshire, UK",26-03-2020,The amazing and have managed to get production...,1
24592,33890,78842,Unknown,06-04-2020,As ridiculous as this post is... We found some...,1


In [30]:
a = list(train[(train["UserName"] == 30510) & (train["ScreenName"] == 75462)]["OriginalTweet"])[0]
a

'Standing in a 40 minute queue for the supermarket and thinking isnÃ\x82Â\x92t it amazing to be outside! Highlight of my day... #coronavirus #lockdown'

In [31]:
print(tv.transform([a]))
print(tv.get_feature_names_out()[tv.transform([a]).argmax()])

  (0, 42351)	0.2851223738998151
  (0, 41557)	0.12755100190282836
  (0, 40938)	0.30689800182967686
  (0, 37226)	0.29112521399940267
  (0, 34982)	0.2645488975981064
  (0, 33093)	0.33871438496962336
  (0, 30402)	0.30375304682886317
  (0, 28944)	0.354372346368912
  (0, 23702)	0.20662775584607432
  (0, 18544)	0.28759755993549146
  (0, 10878)	0.3008137960548647
  (0, 9724)	0.1748073136069567
  (0, 4952)	0.2579468652236513
  (0, 1740)	0.0891401418809861
highlight


**Ответ:** # Очень хороший пример - tf-idf выделит как самое важный токен слово highlight, которое чуть менее очевидно, чем good / great / amazing, но тоже подчеркивает позитивность эмоциональной окраски твита

## Задание 4 Обучение первых моделей (1 балл)

Примените оба векторайзера для получения матриц с признаками текстов.  Выделите целевую переменную.

In [32]:
cvft_v = cvft.toarray()
cvft_v

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [33]:
tvft_v = tvft.toarray()
tvft_v

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [34]:
y = train["Sentiment"]
y

25621    1
30135    1
28899    1
5989     0
4367     1
        ..
20757    0
32103    1
30403    1
21243    0
2732     1
Name: Sentiment, Length: 23410, dtype: int64

Обучите логистическую регрессию на векторах из обоих векторайзеров. Посчитайте долю правильных ответов на обучающих и тестовых данных. Какой векторайзер показал лучший результат? Что можно сказать о моделях?

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr1 = LogisticRegression()
lr1.fit(cvft_v, y)

lr2 = LogisticRegression()
lr2.fit(tvft_v, y)

KeyboardInterrupt: 

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 5 Стемминг (0.5 балла)

Для уменьшения словаря можно использовать стемминг.

Модифицируйте написанный токенайзер, добавив в него стемминг с использованием SnowballStemmer. Обучите Count- и Tfidf- векторайзеры. Как изменился размер словаря?

In [ ]:
def custom_stem_tokenizer(text):
  # -- YOUR CODE HERE --
  return tokens

In [ ]:
custom_stem_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext adding more words to check stemming')

['sampl', 'text', '@sample_text', '#sampletext', 'ad', 'word', 'check', 'stem']

In [ ]:
cv = CountVectorizer # -- YOUR CODE HERE --

print(len(cv.vocabulary_))

36652


**Ответ** # -- YOUR ANSWER HERE --

Обучите логистическую регрессию с использованием обоих векторайзеров. Изменилось ли качество? Есть ли смысл применять стемминг?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание  6 Работа с частотами (1.5 балла)

Еще один способ уменьшить количество признаков - это использовать параметры min_df и max_df при построении векторайзера  эти параметры помогают ограничить требуемую частоту встречаемости токена в документах.

По умолчанию берутся все токены, которые встретились хотя бы один раз.



Подберите max_df такой, что размер словаря будет 36651 (на 1 меньше, чем было). Почему параметр получился такой большой/маленький?

In [ ]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        max_df=# -- YOUR CODE HERE --
                        ).fit(
                            # -- YOUR CODE HERE --
                            )
print(len(cv_df.vocabulary_))

36651


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Подберите min_df (используйте дефолтное значение max_df) в CountVectorizer таким образом, чтобы размер словаря был 3700 токенов (при использовании токенайзера со стеммингом), а качество осталось таким же, как и было. Что можно сказать о результатах?

In [ ]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        min_df=# -- YOUR CODE HERE --
                        ).fit(
                            # -- YOUR CODE HERE --
                            )
print(len(cv_df.vocabulary_))

3700


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

В предыдущих заданиях признаки не скалировались. Отскалируйте данные (при словаре размера 3.7 тысяч, векторизованные CountVectorizer), обучите логистическую регрессию, посмотрите качество и выведите `berplot` содержащий по 10 токенов, с наибольшим по модулю положительными/отрицательными весами. Что можно сказать об этих токенах?

In [ ]:
from sklearn.preprocessing import StandardScaler
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 7 Другие признаки (1.5 балла)

Мы были сконцентрированы на работе с текстами твиттов и не использовали другие признаки - имена пользователя, дату и местоположение

Изучите признаки UserName и ScreenName. полезны ли они? Если полезны, то закодируйте их, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Изучите признак TweetAt в обучающей выборке: преобразуйте его к типу datetime и нарисуйте его гистограмму с разделением по цвету на оспнове целевой переменной. Полезен ли он? Если полезен, то закодируйте его, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --



Поработайте с признаком Location в обучающей выборке. Сколько уникальных значений?

In [ ]:
# -- YOUR CODE HERE --

Постройте гистограмму топ-10 по популярности местоположений (исключая Unknown)

In [ ]:
# -- YOUR CODE HERE --

Видно, что многие местоположения включают в себя более точное название места, чем другие (Например, у некоторых стоит London, UK; а у некоторых просто UK или United Kingdom).

Создайте новый признак WiderLocation, который содержит самое широкое местоположение (например, из London, UK должно получиться UK). Сколько уникальных категорий теперь? Постройте аналогичную гистограмму.

In [ ]:
# -- YOUR CODE HERE --

Закодируйте признак WiderLocation с помощью OHE таким образом, чтобы создались только столбцы для местоположений, которые встречаются более одного раза. Сколько таких значений?


In [ ]:
# -- YOUR CODE HERE --

Добавьте этот признак к матрице отскалированных текстовых признаков, обучите логистическую регрессию, замерьте качество. Как оно изменилось? Оказался ли признак полезным?


*Подсказка:* используйте параметр `categories` в энкодере.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 8 Хорошее качество (Бонус 1 балл)

Добейтесь accuracy=0.9 на тестовой выборке (можно сменить токенайзер, векторайзер, модель и т.д.)

In [ ]:
# -- YOUR CODE HERE --